*Enter your observations here*

In [1]:
import mailbox
import csv
from bs4 import BeautifulSoup as bs
from bs4 import Comment
import re
import pandas as pd
import numpy as np
import time
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA





stop_words = set(stopwords.words('english'))
print(stop_words)

def createCSV(name, mbox, mes_type):
    #createing SPAM csv
    writer = csv.writer(open(name, "w", encoding="utf-8"))
    writer.writerow(['Subject', 'From', 'Date', 'Message-ID', 'Payload', 'Type'])
    for message in mailbox.mbox(mbox):

        if message.is_multipart():
            content = ''.join(str(part.get_payload(decode=True)) for part in message.get_payload())
            content = cleanMe(content)
        else:
            content = str(message.get_payload(decode=True))
            content = cleanMe(content)
#         content = content.split()
        #print(content)
        writer.writerow([message['subject'], message['from'], message['date'], message['Message-Id'], content, mes_type])
    

{'then', 'over', 'having', 'just', 'theirs', 'before', 'those', 'now', 'that', 'she', 'll', 'ain', 'whom', "hadn't", 'yourselves', 'for', 'again', 'themselves', 'didn', 'with', 've', "should've", 'are', 'these', 'which', 'here', "shouldn't", 'as', 'your', 'a', 'not', 'doesn', 'but', 'into', 'wouldn', 'both', 'i', 'did', 'further', 'does', 'each', 'very', "hasn't", "it's", 'weren', 'was', 'can', "wasn't", 'd', 'you', "you've", 'our', 'couldn', 'this', 'am', 'yourself', 'too', 'aren', "haven't", 'don', 'yours', 'himself', "weren't", 'against', 'once', 'hasn', "mustn't", 'if', 'up', 're', 'under', "isn't", 'while', 'from', 'have', 'than', 'me', 'were', 'more', 'same', 'of', 'so', 'own', 'any', 'm', 'should', 'shouldn', 'hers', 'how', 'what', 'off', 'why', "you'd", 'about', 'be', 'my', 'an', 'to', 'needn', 'itself', "doesn't", 'the', 'or', 'after', 'ourselves', 's', 'during', 'between', 'until', 'his', 'there', 'few', 't', 'on', 'and', 'other', 'above', 'do', 'down', "don't", 'where', 'the

In [2]:
def cleanMe(html):
    soup = bs(html,"lxml") # create a new bs4 object from the html data loaded
    for script in soup(["script", "style"]): # remove all javascript and stylesheet code
        script.extract()
    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n '.join(chunk for chunk in chunks if chunk)
    text = text.replace('\\n', ' ').replace('\\t', ' ').replace("\n", ' ').replace('\\b', ' ')
    # gets rid of escape characters
    text = bytes(text, "utf-8").decode("unicode_escape")
    # gets rid of links 
    text = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)
    text = " ".join(re.findall(r'\s+[a-zA-Z]+\s+', text))
    # make text lower case
    text = text.lower()
    text = re.sub(r'\s+', ' ', text)
    # remove stop words
    resultwords  = [word for word in text.split() if word not in stop_words]
    result = ' '.join(resultwords)
    return result

In [3]:
#createCSV('./data/spam_no_stop.csv', './data/Spam.mbox', 'Spam')
#createCSV('./data/inbox_no_stop.csv', './data/Inbox.mbox', 'Ham')

In [4]:
spam1 = pd.read_csv("./data/spam1.csv")
print(spam1.shape)
inbox1 = pd.read_csv("./data/inbox.csv")
print(inbox1.shape[0])


(707, 6)
51112


In [6]:
# inbox concatenated vectors = icv
icv_id = []
icv_word = []
icv_target = []

spam_start = time.time()
for index, row in spam1.iterrows():
    email_words = str(row['Payload']).strip()
    unique_id = row["Message-ID"]
    words_dict = {}
    words_vec = email_words.split(" ")
    for word in words_vec:
        words_dict[word] = True
    for word in words_dict.keys():
        icv_id.append(unique_id)
        icv_word.append(word)
        icv_target.append(1)

print("Spam Time taken: ", str(time.time() - spam_start))

inbox_start = time.time()
for index, row in inbox1.iterrows():
    email_words = str(row['Payload']).strip()
    unique_id = row["Message-ID"]
    words_dict = {}
    words_vec = email_words.split(" ")
    for word in words_vec:
        words_dict[word] = True
    for word in words_dict.keys():
        icv_id.append(unique_id)
        icv_word.append(word)
        icv_target.append(0)

print("Ham Time taken: ", str(time.time() - inbox_start))
            
new_df = pd.DataFrame({"id":icv_id, "word": icv_word, "target": icv_target})


Spam Time taken:  0.1002049446105957
Ham Time taken:  6.941816806793213


In [7]:
word_spam_df = new_df
word_spam_df_features = word_spam_df.loc[:, "word"]

word_spam_df_outcomes = word_spam_df.loc[:, "target"]
word_spam_df.shape
word_spam_df
# tyring to take out stop words after df is made. didn't do 
df = word_spam_df[word_spam_df.word != stop_words]
df.shape

(3850402, 3)

In [8]:
total_ham = inbox1.shape[0]
total_spam = spam1.shape[0]
total = total_spam + total_ham
p_ham = total_spam / total
p_spam = total_ham / total
print(p_ham, p_spam)


0.013643644223161389 0.9863563557768386


In [10]:
def getUniqueWordCounts(word_and_spam, ham_total, spam_total, ham_p, spam_p):
    seen_words = {}
    word_probs = []
    
    unique_words = word_and_spam.word.unique()
    unique_words_for_priors = []
    unique_word_counts = {}
    for w in unique_words:
        unique_word_counts[w] = {"s":0, "h":0}
       
    for index, row in word_and_spam.iterrows():
        word = row['word']
        if (word not in seen_words):
            if (row["target"] == 1):
                unique_word_counts[word]["s"] = unique_word_counts[word]["s"] + 1
            else:
                unique_word_counts[word]["h"] = unique_word_counts[word]["h"] + 1
                
#             w_r = word_and_spam[word_and_spam["word"] == word]
#             spam_word_count = len(w_r[(word_and_spam["target"] == 1)])
#             ham_word_count = len(w_r[(word_and_spam["target"] == 0)])
#             p_spam_word = spam_word_count / spam_total
#             p_ham_word = ham_word_count / ham_total
#             p_s = p_spam_word * spam_p
#             p = (p_s)  / (p_s + p_ham_word * ham_p)
#             word_probs.append({"key": word, "val": p})
            
#     spam_status_for_unique_words = []
#     for w in unique_words:
#         spam_status_for_unique_words.append(0)
#         spam_status_for_unique_words.append(1)
#         unique_words_for_priors.append(w)
#         unique_words_for_priors.append(w)
        
    # these could also perhaps be total spam or ham emails?
#     for index, row in word_and_spam.iterrows():
#         word = row['word']
#         if (word not in seen_words):
#             seen_words[word] = True
#             w_r = word_and_spam[word_and_spam["word"] == word]
#             spam_word_count = len(w_r[(word_and_spam["target"] == 1)])
#             ham_word_count = len(w_r[(word_and_spam["target"] == 0)])
#             p_spam_word = spam_word_count / spam_total
#             p_ham_word = ham_word_count / ham_total
#             p_s = p_spam_word * spam_p
#             p = (p_s)  / (p_s + p_ham_word * ham_p)
#             word_probs.append({"key": word, "val": p})
    return unique_word_counts


counts_df = getUniqueWordCounts(word_spam_df, total_ham, total_spam, p_ham, p_spam)

In [17]:
# counts_df_not_dict = pd.DataFrame(counts_df)
total_ham = inbox1.shape[0]
total_spam = spam1.shape[0]
total = total_spam + total_ham
p_ham = total_spam / total
p_spam = total_ham / total
print(p_ham, p_spam)

word_arr = []
target_arr = []
p_arr = []
for key, val in counts_df.items():
    spam_word_count = val["s"]
    ham_word_count = val["h"]
    
    p_word_given_spam = spam_word_count / total_spam
    p_word_given_ham = ham_word_count / total_ham    
    p_s = p_word_given_spam * p_spam
    p_h = p_word_given_ham * p_ham
    
    p_spam_given_word = (p_s)  / (p_s + p_h)
    p_ham_given_word = (p_h)  / (p_s + p_h)
    
    word_arr.append(key)
    target_arr.append(1)
    p_arr.append(p_spam_given_word)
    word_arr.append(key)
    target_arr.append(0)
    p_arr.append(p_ham_given_word)

    
final_df = pd.DataFrame(data={"word":word_arr,"target":target_arr,"p":p_arr})

0.013643644223161389 0.9863563557768386


In [18]:
pd.to_csv(data=final_df, )


,p,target,word
0,0.985723,1,happy
1,0.014277,0,happy
2,0.972408,1,email
3,0.027592,0,email
4,0.981502,1,displaying
5,0.018498,0,displaying
6,0.957578,1,view
7,0.042422,0,view
8,0.979036,1,in
9,0.020964,0,in


In [13]:
counts_df_not_dict

,a,aa,aaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaarjlbasmw,aaaaaaabaaidbaugbwgjcgsqaaeeaqmcbaifbwyibqmmmweaahedbcesmqvbuwetingbmgyukagx,aaaaaaabaaidbaugbwgjcgsqaaibawmcbaigbwmeagyccwecaxeeaauhejfbuqytysjxgrqykaeh,aaaaaall,aaaaand,aaacoppertop,...,zzq,zzqj,zzqqwos,zzt,zzu,zzug,zzv,zzy,zzym,zzzup
h,26196,143,35,1,1,1,2,1,1,2,...,3,1,1,2,2,2,1,2,1,1
s,286,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# didn't get time to write this...but I think the getProirs dictionary could get changed to a matrix easily
#matrixPrior(words_spam):
#    for 

In [3]:
from sklearn import naive_bayes as nb
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(word_spam_df_features, word_spam_df_outcomes, random_state=12345, test_size=0.33)


NameError: name 'word_spam_df_features' is not defined

In [4]:
nb_model_1 = nb.GaussianNB().fit(X_train, y_train)
nb_model_1.score(X_test, y_test)

NameError: name 'X_train' is not defined

In [12]:
# parameters for pipeline
from sklearn.ensemble import RandomForestClassifier
# random forest
rfc = RandomForestClassifier()
param_grid_rfc = {'randomforestclassifier__n_estimators': np.arange(1,10)}

# niave bayse
clf = GaussianNB()
param_grid_rfc = {'gaussiannb__prior': True}

NameError: name 'GaussianNB' is not defined

In [7]:
def run_model(model, param_grid, xtrain, ytrain, do_pca = False):
    scaler = MinMaxScaler()
    if(do_pca == True):
        pca = PCA(n_components = 10)
        pipe = make_pipeline(pca, model)
    else:
        pipe = make_pipeline(model)
    grid = GridSearchCV(pipe,param_grid)
    grid.fit(xtrain, ytrain)
    grid.best_params_
    accuracy = grid.score(xtrain, ytrain)
    print(f"In-sample accuracy: {accuracy:0.2%}")
    return(grid)

In [ ]:
# add 
